In [ ]:
#!/usr/bin/env python3
import os
import json
import h5py
import argparse
import pandas as pd
import numpy as np
import tinydb as db
from tinydb.storages import MemoryStorage
import matplotlib as mpl
import matplotlib.pyplot as plt
# plt.style.use('../clint.mpl')
from matplotlib.colors import LogNorm

import boost_histogram as bh
import pickle as pl

from pygama import DataGroup
import pygama.lh5 as lh5
import pygama.analysis.histograms as pgh
import pygama.analysis.peak_fitting as pgf
import cage_utils
plt.rcParams['figure.figsize'] = [14, 10]

In [ ]:
run = 60 #44 #70 #64
dsp_id = '02'
user = True
hit =True
cal = True
lowE=False
etype = 'trapEftp_cal'

corr_DCR=True
corr_AoE=True
corr_ToE=True

norm = True
cut = True


dsp_list = ['energy', 'trapEftp', 'trapEmax', 'trapEftp_cal', 'bl','bl_sig', 'bl_slope', 'AoE', 'dcr', "tp_0", "tp_02", "tp_05", "tp_10", "tp_20", "tp_30", "tp_40", "tp_50", "tp_60", "tp_70", "tp_80", "tp_90", "tp_96", 'tp_max', 'ToE', 'log_tail_fit_slope', 'wf_max', 'wf_argmax', 'trapE_argmax', 'lf_max']

df_raw, dg, runtype, rt_min, radius, angle_det, rotary = cage_utils.getDataFrame(run, user=user, hit=hit, cal=cal, dsp_list=dsp_list, lowE=lowE)

df = cage_utils.apply_DC_Cuts(run, df_raw)

df_60 = df.query('trapEftp_cal > 55 and trapEftp_cal < 65').copy()


if corr_ToE==True:
    ToE_mode = cage_utils.mode_hist(df, param='ToE', a_bins=1000, alo=-0.1, ahi=0.45, cut=False, cut_str='')
    df['ToE_plot'] = df['ToE'] - ToE_mode
    df_60['ToE_plot'] = df_60['ToE'] - ToE_mode

In [ ]:
blo, bhi, bpb = -5., 5., 0.005
nbx = int((bhi-blo)/bpb)

bl_hist, bins = np.histogram(df['bl_slope'], bins=nbx,
                range=[blo, bhi])

plt.semilogy(bins[1:], bl_hist, ds='steps', c='b', lw=1)


plt.axvline(bl_slope_lo, c='r', lw=1)
plt.axvline(bl_slope_hi, c='r', lw=1)

plt.xlabel('bl', ha='right', x=1)
plt.ylabel('counts', ha='right', y=1)
plt.show()

In [ ]:
norm=False
fig, ax = plt.subplots()
elo, ehi, epb = 0, 2000, 0.5

# elo, ehi, epb = 0, 10000, 10
emax_lo, emax_hi, emax_pb= 0, 6000, 10

nbx = int((ehi-elo)/epb)
nby = int((emax_hi-emax_lo)/emax_pb)


if norm==True:
    rt = np.array([(1/rt_min)])
    wts = np.repeat(rt, len(df_60['trapEftp_cal']))
else:
    rt = np.array([(1/1.)])
    wts = np.repeat(rt, len(df_60['trapEftp_cal']))


# fig.suptitle(f'T/E vs Energy', horizontalalignment='center', fontsize=16)

noise_hist_norm, xedges, yedges = np.histogram2d(df_60['trapEftp_cal'], df_60['trapEmax'], bins=[nbx, nby], range=([elo, ehi], [emax_lo, emax_hi]), weights=wts)
X, Y = np.mgrid[elo:ehi:nbx*1j, emax_lo:emax_hi:nby*1j]


pcm = plt.pcolormesh(X, Y, noise_hist_norm, norm=LogNorm()) #0.002, 0.2

cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)
ax.set_xlabel(f'trapEftp', fontsize=16)
ax.set_ylabel('trapEmax)', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=14)
plt.setp(ax.get_yticklabels(), fontsize=14)


ax.text(0.95, 0.83, f'r = {radius} mm \ntheta = {angle_det} deg', verticalalignment='bottom',
            horizontalalignment='right', transform=ax.transAxes, color='green', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

# plt.legend()
plt.title(f'\n{runtype} run {run}, {rt_min:.2f} mins', fontsize=12)
plt.tight_layout()
# plt.ylim(-0.25, 0.09)
# plt.ylim(0, 0.1)
# plt.xlim(52.5, 65)
plt.xlim(50, 100)
plt.ylim(0, 1000)
plt.show()

In [ ]:
norm=True
fig, ax = plt.subplots()
elo, ehi, epb = 40, 80, 0.5
ToElo, ToEhi, ToEpb= -0.2, 0.2, 0.0025

nbx = int((ehi-elo)/epb)
nby = int((ToEhi-ToElo)/ToEpb)


if norm==True:
    rt = np.array([(1/rt_min)])
    wts = np.repeat(rt, len(df['trapEftp_cal']))
else:
    rt = np.array([(1/1.)])
    wts = np.repeat(rt, len(df['trapEftp_cal']))


fig.suptitle(f'T/E vs Energy', horizontalalignment='center', fontsize=16)

ToE_hist_norm, xedges, yedges = np.histogram2d(df['trapEftp_cal'], df['ToE_plot'], bins=[nbx, nby], range=([elo, ehi], [ToElo, ToEhi]), weights=wts)
X, Y = np.mgrid[elo:ehi:nbx*1j, ToElo:ToEhi:nby*1j]


pcm = plt.pcolormesh(X, Y, ToE_hist_norm, norm=LogNorm(0.001, 1)) #0.002, 0.2

cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)
ax.set_xlabel(f'Energy (keV)', fontsize=16)
ax.set_ylabel('T/E (arb)', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=14)
plt.setp(ax.get_yticklabels(), fontsize=14)


ax.text(0.95, 0.83, f'r = {radius} mm \ntheta = {angle_det} deg \nwf_triangle 100 ns, 10 ns', verticalalignment='bottom',
            horizontalalignment='right', transform=ax.transAxes, color='green', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

# plt.legend()
plt.title(f'\n{runtype} run {run}, {rt_min:.2f} mins', fontsize=12)
plt.tight_layout()
# plt.ylim(-0.05, 0.1)
# plt.ylim(-0.05, 0.08)
plt.ylim(-0.2, 0.2)
# plt.xlim(52.5, 65)
plt.show()

In [ ]:
# cut for all 60 keV 
# run 64

cut_60 = f'ToE_plot > 0.0325 and ToE_plot < 0.085 and trapEftp_cal >=58.0 and trapEftp_cal <= 61.5'
times, superpulse_60 = cage_utils.get_superpulse(df_60, dg, cut_60, all=True, norm=True)



In [ ]:
cut_bkg_low = f'trapEftp_cal >=56.25 and trapEftp_cal < 58.0'
cut_bkg_hi = f'trapEftp_cal >61.5 and trapEftp_cal <= 63.25'
cut_bkg = [cut_bkg_low, cut_bkg_hi]
print(cut_bkg)
times, window_wfs, bkg_superpulse = cage_utils.get_superpulse_window(df, dg, cut_bkg, all=True, norm=True)

In [ ]:
print(len(df))

In [ ]:

df_60_low = df.query(cut_60).copy()
# times, wfs_60 = cage_utils.get_wfs(df, dg, cut_bkg_low, all=True)

In [ ]:
# Make energy spectrum
elo, ehi, epb = 80000, 120000, 100
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)

energy_hist, bins = np.histogram(df['energy'], bins=nbx,
                range=[elo, ehi])

plt.semilogy(bins[1:], energy_hist, ds='steps', c='b', lw=1) #, label=f'{etype}'

# plt.xlabel(etype, ha='right', x=1)
# plt.ylabel('cts / sec', ha='right', y=1)
# # plt.legend()
# # plt.title(f'{etype}: {runtype} run {run} , {rt_min:.2f} mins')
# plt.savefig(f'./plots/normScan/{runtype}_energy_run{run}.pdf', dpi=200)
# plt.xlim(80000, 150000)
# plt.xlim(80000, 1500000)
plt.show()

In [ ]:
# Make energy spectrum
elo, ehi, epb = 0, 30000000, 100
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)

energy_hist, bins = np.histogram(df_60['energy'], bins=nbx,
                range=[elo, ehi])

plt.semilogy(bins[1:], energy_hist, ds='steps', c='b', lw=1) #, label=f'{etype}'

# plt.xlabel(etype, ha='right', x=1)
# plt.ylabel('cts / sec', ha='right', y=1)
# # plt.legend()
# # plt.title(f'{etype}: {runtype} run {run} , {rt_min:.2f} mins')
# plt.savefig(f'./plots/normScan/{runtype}_energy_run{run}.pdf', dpi=200)
plt.xlim(80000, 120000)
plt.show()

In [ ]:
# Make energy spectrum
elo, ehi, epb = 50, 80, 0.5
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)

energy_hist, bins = np.histogram(df_60['trapEftp_cal'], bins=nbx,
                range=[elo, ehi])

plt.semilogy(bins[1:], energy_hist, ds='steps', c='b', lw=1) #, label=f'{etype}'

# plt.xlabel(etype, ha='right', x=1)
# plt.ylabel('cts / sec', ha='right', y=1)
# # plt.legend()
# # plt.title(f'{etype}: {runtype} run {run} , {rt_min:.2f} mins')
# plt.savefig(f'./plots/normScan/{runtype}_energy_run{run}.pdf', dpi=200)
# plt.xlim(80000, 150000)
plt.show()

In [ ]:
# Make energy spectrum
elo, ehi, epb = 0, 300, 1
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)

energy_hist, bins = np.histogram(df_60['trapEftp'], bins=nbx,
                range=[elo, ehi])

plt.semilogy(bins[1:], energy_hist, ds='steps', c='b', lw=1) #, label=f'{etype}'

# plt.xlabel(etype, ha='right', x=1)
# plt.ylabel('cts / sec', ha='right', y=1)
# # plt.legend()
# # plt.title(f'{etype}: {runtype} run {run} , {rt_min:.2f} mins')
# plt.savefig(f'./plots/normScan/{runtype}_energy_run{run}.pdf', dpi=200)
# plt.xlim(80000, 150000)
plt.xlim(0, 300)
plt.show()

In [ ]:
# print(df.query(cut_bkg_hi))
# print(df.query(cut_bkg_low))
# print(len(df.query(cut_bkg_hi).copy()))
# print(df.query(cut_bkg_hi).copy()['trapEftp_cal'])
new_df = df.query(cut_bkg_low).copy()
# en = np.array(new_df['trapEftp_cal'])
# print(np.amax(en))
en = np.array(new_df['energy'])
print(np.amax(en))
# print(new_df['trapEftp_cal'].min)

In [ ]:
12018140

In [ ]:
print(bkg_superpulse)

In [ ]:
fig, ax = plt.subplots()
plt.plot(times, window_wfs[0][:len(bkg_superpulse)-1], '-b', label=('background: '+ cut_bkg_low ))
plt.plot(times, window_wfs[1][:len(bkg_superpulse)-1], '-g', label=('background: '+ cut_bkg_hi))
plt.legend()
plt.xlim(3700, 4300)
# plt.ylim(-2, 15)
plt.title(f'run {run}; {radius} mm')
plt.show()

In [ ]:
t, window_2_wfs = cage_utils.get_wfs(df_60, dg, cut_bkg_hi, all=True)

In [ ]:
t, window_1_wfs = cage_utils.get_wfs(df_60, dg, cut_bkg_low, all=True)

In [ ]:
print(len(window_2_wfs))

In [ ]:
# fig, ax = plt.subplots(figsize=(9,8))
ax = plt.axes()

# set up colorbar to plot waveforms of different energies different colors
colors = plt.cm.viridis(np.linspace(0, 1, len(window_1_wfs)))
c = np.arange(0, len(window_1_wfs))
norm = mpl.colors.Normalize(vmin=c.min(), vmax=c.max())
cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.plasma)
cmap.set_array([])

for n in range(len(window_1_wfs)):
    plt.plot(t, window_1_wfs[n][:len(window_1_wfs[n])-1], c=cmap.to_rgba(n))

# cb = fig.colorbar(cmap, ticks=(57.5, 59.5))
cb.set_label("Energy", ha = 'right', va='center', rotation=270, fontsize=20)
cb.ax.tick_params(labelsize=18)

ax.text(0.95, 0.83, f'{cut_bkg_hi}', verticalalignment='top',
                        horizontalalignment='right', transform=ax.transAxes, color='green', fontsize=14,
                        bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

plt.title(f'run {run}: r = {radius} mm \ntheta = {angle_det} deg')

#     plt.xlim(3800, 8000)
#     plt.ylim(0.4, 1.01)
plt.setp(ax.get_xticklabels(), fontsize=16)
plt.setp(ax.get_yticklabels(), fontsize=16)
plt.title(f'60 keV waveforms for run {run}\nr = {radius} mm; theta = {angle_det} deg', fontsize=20)
plt.xlabel('clock cycles', fontsize=20)
# plt.xlim(3900, 4050)

In [ ]:
# fig, ax = plt.subplots(figsize=(9,8))
ax = plt.axes()

# set up colorbar to plot waveforms of different energies different colors
colors = plt.cm.viridis(np.linspace(0, 1, len(window_2_wfs)))
c = np.arange(0, len(window_2_wfs))
norm = mpl.colors.Normalize(vmin=c.min(), vmax=c.max())
cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.plasma)
cmap.set_array([])

for n in range(len(window_2_wfs)):
    plt.plot(t, window_2_wfs[n][:len(window_2_wfs[n])-1], c=cmap.to_rgba(n))

# cb = fig.colorbar(cmap, ticks=(57.5, 59.5))
cb.set_label("Energy", ha = 'right', va='center', rotation=270, fontsize=20)
cb.ax.tick_params(labelsize=18)

ax.text(0.95, 0.83, f'{cut_bkg_hi}', verticalalignment='top',
                        horizontalalignment='right', transform=ax.transAxes, color='green', fontsize=14,
                        bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

plt.title(f'run {run}: r = {radius} mm \ntheta = {angle_det} deg')

#     plt.xlim(3800, 8000)
#     plt.ylim(0.4, 1.01)
plt.setp(ax.get_xticklabels(), fontsize=16)
plt.setp(ax.get_yticklabels(), fontsize=16)
plt.title(f'60 keV waveforms for run {run}\nr = {radius} mm; theta = {angle_det} deg', fontsize=20)
plt.xlabel('clock cycles', fontsize=20)
# plt.xlim(3900, 4050)
# plt.ylim(-25, 250)

In [ ]:
fig, ax = plt.subplots()
plt.plot(times, superpulse_60[:len(superpulse_60)-1], '-r', label=cut_60)
plt.plot(times, bkg_superpulse[:len(bkg_superpulse)-1], '-b', label=('background: '+ cut_bkg_low + cut_bkg_hi))
plt.legend()
plt.xlim(3700, 4300)
# plt.ylim(-2, 15)
plt.title(f'run {run}; {radius} mm')
plt.show()

In [ ]:
# def get_wfs(df, dg, cut_str='', nwfs = 10):
#     idx = df.query(cut_str).index[:nwfs]
#     raw_store = lh5.Store()
#     tb_name = 'ORSIS3302DecoderForEnergy/raw'
#     lh5_dir = dg.lh5_dir
#     raw_list = lh5_dir + dg.fileDB['raw_path'] + '/' + dg.fileDB['raw_file']
#     print(raw_list.values)
#     if len(raw_list)>2:
#         raw_int = int(len(raw_list)/2) -1
#     else:
#         raw_int = len(raw_list)-1
#     f_raw = raw_list.values[raw_int] # fixme, only works for one file rn
#     #f_raw = raw_list.values[0] # fixme, only works for one file rn
#     print(f_raw)
#     data_raw, nrows = raw_store.read_object(tb_name, f_raw, start_row=0, n_rows=idx[-1]+1)

#     wfs_all = (data_raw['waveform']['values']).nda
#     wfs = wfs_all[idx.values, :]
#     # baseline subtraction
#     bl_means = wfs[:,:800].mean(axis=1)
#     wf_blsub = (wfs.transpose() - bl_means).transpose()
#     ts = np.arange(0, wf_blsub.shape[1]-1, 1)
    
#     return(ts, wf_blsub)

In [ ]:
# run 64

cut = f'ToE_plot > 0.045 and ToE_plot < 0.075 and trapEftp_cal >58.0 and trapEftp_cal < 59.5'
times, superpulse = cage_utils.get_superpulse(df, dg, cut, all=True, norm=False)
#t, wfs = cage_utils.get_wfs(df, dg, cut, nwfs=5)

cut2 = f'ToE_plot > 0.0325 and ToE_plot < 0.0625 and trapEftp_cal >59.5 and trapEftp_cal < 61.0'
times, superpulse2 = cage_utils.get_superpulse(df, dg, cut2, all=True, norm= False)
#t, wfs2 = cage_utils.get_wfs(df, dg, cut2, nwfs=5)

cut3 = f'trapEftp_cal >55.5 and trapEftp_cal < 57.0'
times, bkg_superpulse = cage_utils.get_superpulse(df, dg, cut3, all=True, norm=False)

In [ ]:
#get and draw all waveforms individually

t, wfs = cage_utils.get_wfs(df, dg, cut, all=True)

t, wfs2 = cage_utils.get_wfs(df, dg, cut2, all=True)

In [ ]:
t, wfs3 = cage_utils.get_wfs(df, dg, cut3, all=True)

In [ ]:
# # run 44

# cut = f'ToE_plot > 0.025 and ToE_plot < 0.065 and trapEftp_cal >58 and trapEftp_cal < 59.5'
# times, superpulse = cage_utils.get_superpulse(df, dg, cut)
# t, wfs = cage_utils.get_wfs(df, dg, cut, nwfs=5)

# cut2 = f'ToE_plot > 0.01 and ToE_plot < 0.05 and trapEftp_cal >59.5 and trapEftp_cal < 61.5'
# times, superpulse2 = cage_utils.get_superpulse(df, dg, cut2)
# t, wfs2 = cage_utils.get_wfs(df, dg, cut2, nwfs=5)

# cut3 = f'trapEftp_cal >55.5 and trapEftp_cal < 57.5'
# times, bkg_superpulse = cage_utils.get_superpulse(df, dg, cut3)

# run 42

# cut = f'ToE_plot > 0.04 and ToE_plot < 0.08 and trapEftp_cal >58 and trapEftp_cal < 59.5'
# times, superpulse = cage_utils.get_superpulse(df, dg, cut, all = True)
# t, wfs = cage_utils.get_wfs(df, dg, cut, nwfs=5)

# cut2 = f'ToE_plot > 0.03 and ToE_plot < 0.06 and trapEftp_cal >59.5 and trapEftp_cal < 61.5'
# times, superpulse2 = cage_utils.get_superpulse(df, dg, cut2, all = True)
# t, wfs2 = cage_utils.get_wfs(df, dg, cut2, nwfs=5)

# cut3 = f'trapEftp_cal >55.5 and trapEftp_cal < 57.5'
# times, bkg_superpulse = cage_utils.get_superpulse(df, dg, cut3, all = True)

In [ ]:
fig, ax = plt.subplots()
plt.plot(times, superpulse[:len(superpulse)-1], '-r', label=cut) #label='58 - 59.5 keV'
plt.plot(times, superpulse2[:len(superpulse)-1], '-g', label=cut2) #label='59.5 - 61 keV'
plt.plot(times, bkg_superpulse[:len(superpulse)-1], '-b', label=('background: '+ cut3), linewidth=0.5)
plt.legend()
# plt.xlim(3600, 4500)
plt.title(f'run {run}; {radius} mm')
plt.show()


In [ ]:
fig, ax = plt.subplots()
plt.plot(times, superpulse[:len(superpulse)-1], '-r', label=cut)
plt.plot(times, superpulse2[:len(superpulse)-1], '-g', label=cut2)
plt.plot(times, bkg_superpulse[:len(superpulse)-1], '-b', label=('background: '+ cut3))
plt.legend()
plt.xlim(3900, 4100)
# plt.ylim(-2, 15)
plt.title(f'run {run}; {radius} mm')
plt.show()

In [ ]:
fig, ax = plt.subplots()
plt.plot(times, superpulse[:len(superpulse)-1], '-r', label=cut)
plt.legend()
plt.title(f'run {run}; {radius} mm')
plt.show()

In [ ]:
fig, ax = plt.subplots()
# plt.plot(times, superpulse[:len(superpulse)-1], '-r')
plt.plot(times, superpulse2[:len(superpulse)-1], '-g', label=cut2)
plt.legend()
plt.title(f'run {run}; {radius} mm')
plt.show()

In [ ]:
fig, ax = plt.subplots()
# plt.plot(times, superpulse[:len(superpulse)-1], '-r')

plt.plot(t, wfs[0][:len(wfs[0])-1], '-g', label=cut)
plt.plot(t, wfs2[0][:len(wfs2[0])-1], '-r', label=cut2)
plt.legend()
plt.title(f'run {run}; {radius} mm')
plt.xlim(3700, 4200)
plt.show()

In [ ]:
# fig, ax = plt.subplots(figsize=(9,8))
ax = plt.axes()

# set up colorbar to plot waveforms of different energies different colors
colors = plt.cm.viridis(np.linspace(0, 1, len(wfs)))
c = np.arange(0, len(wfs))
norm = mpl.colors.Normalize(vmin=c.min(), vmax=c.max())
cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.plasma)
cmap.set_array([])

for n in range(len(wfs)):
    plt.plot(t, wfs[n][:len(wfs[n])-1], c=cmap.to_rgba(n))

# cb = fig.colorbar(cmap, ticks=(57.5, 59.5))
cb.set_label("Energy", ha = 'right', va='center', rotation=270, fontsize=20)
cb.ax.tick_params(labelsize=18)

ax.text(0.95, 0.83, f'{cut}', verticalalignment='top',
                        horizontalalignment='right', transform=ax.transAxes, color='green', fontsize=14,
                        bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

plt.title(f'run {run}: r = {radius} mm \ntheta = {angle_det} deg')

#     plt.xlim(3800, 8000)
#     plt.ylim(0.4, 1.01)
plt.setp(ax.get_xticklabels(), fontsize=16)
plt.setp(ax.get_yticklabels(), fontsize=16)
plt.title(f'60 keV waveforms for run {run}\nr = {radius} mm; theta = {angle_det} deg', fontsize=20)
plt.xlabel('clock cycles', fontsize=20)
plt.xlim(3900, 4050)
plt.ylim(-25, 250)

In [ ]:
# fig, ax = plt.subplots(figsize=(9,8))
ax = plt.axes()

# set up colorbar to plot waveforms of different energies different colors
colors = plt.cm.viridis(np.linspace(0, 1, len(wfs2)))
c = np.arange(0, len(wfs2))
norm = mpl.colors.Normalize(vmin=c.min(), vmax=c.max())
cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.plasma)
cmap.set_array([])

for n in range(len(wfs2)):
    plt.plot(t, wfs2[n][:len(wfs2[n])-1], c=cmap.to_rgba(n))

# cb = fig.colorbar(cmap, ticks=(57.5, 59.5))
cb.set_label("Energy", ha = 'right', va='center', rotation=270, fontsize=20)
cb.ax.tick_params(labelsize=18)

ax.text(0.95, 0.83, f'{cut2}', verticalalignment='top',
                        horizontalalignment='right', transform=ax.transAxes, color='green', fontsize=14,
                        bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

#     plt.xlim(3800, 8000)
#     plt.ylim(0.4, 1.01)
plt.setp(ax.get_xticklabels(), fontsize=16)
plt.setp(ax.get_yticklabels(), fontsize=16)
plt.title(f'60 keV waveforms for run {run}\nr = {radius} mm; theta = {angle_det} deg', fontsize=20)
plt.xlabel('clock cycles', fontsize=20)
plt.xlim(3900, 4050)
plt.ylim(-25, 250)

In [ ]:
# fig, ax = plt.subplots(figsize=(9,8))
ax = plt.axes()

# set up colorbar to plot waveforms of different energies different colors
colors = plt.cm.viridis(np.linspace(0, 1, len(wfs3)))
c = np.arange(0, len(wfs3))
norm = mpl.colors.Normalize(vmin=c.min(), vmax=c.max())
cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.plasma)
cmap.set_array([])

for n in range(len(wfs3)):
    plt.plot(t, wfs3[n][:len(wfs3[n])-1], c=cmap.to_rgba(n))

# cb = fig.colorbar(cmap, ticks=(57.5, 59.5))
cb.set_label("Energy", ha = 'right', va='center', rotation=270, fontsize=20)
cb.ax.tick_params(labelsize=18)

ax.text(0.95, 0.83, f'{cut3}', verticalalignment='top',
                        horizontalalignment='right', transform=ax.transAxes, color='green', fontsize=14,
                        bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

#     plt.xlim(3800, 8000)
#     plt.ylim(0.4, 1.01)
plt.setp(ax.get_xticklabels(), fontsize=16)
plt.setp(ax.get_yticklabels(), fontsize=16)
plt.title(f'60 keV waveforms for run {run}\nr = {radius} mm; theta = {angle_det} deg', fontsize=20)
plt.xlabel('clock cycles', fontsize=20)
plt.xlim(3900, 4050)
# plt.ylim(-25, 250)

In [ ]:
# fig, ax = plt.subplots(figsize=(9,8))
ax = plt.axes()


for n in range(len(wfs)):
    plt.plot(t, wfs[n][:len(wfs[n])-1], '-r', label=cut)
    
for m in range(len(wfs)):
    plt.plot(t, wfs2[m][:len(wfs2[m])-1], '-g', label=cut2)

cb.set_label("Energy", ha = 'right', va='center', rotation=270, fontsize=20)
cb.ax.tick_params(labelsize=18)

#     plt.xlim(3800, 8000)
#     plt.ylim(0.4, 1.01)
plt.setp(ax.get_xticklabels(), fontsize=16)
plt.setp(ax.get_yticklabels(), fontsize=16)
plt.title(f'Waveforms', fontsize=20)
plt.xlabel('clock cycles', fontsize=20)
plt.xlim(3900, 4050)
# plt.legend()